In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from itertools import product
from typing import Tuple

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
import seaborn as sns

import sbibm
from sbibm.utils.io import get_tensor_from_csv

import sbi.analysis as analysis

from notebook import nre_gamma_default, wide_to_long

In [3]:
nre_gamma_default = float("Inf")

## what do we expect for experiments

In [4]:
columns = ['task', 'num_observation', 'algorithm', 'num_contrastive_parameters', 'gamma', 'num_blocks']

# general for nre
# tasks = sbibm.get_available_tasks()
tasks = ['slcp', 'two_moons', 'gaussian_mixture']
num_observation = list(range(1,11))
num_contrastive_parameters = [2, 10, 25, 50, 75, 100, 150, 200]
num_blocks = [2, 3]

# for cnre
gammas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

In [5]:
experiments_cnre = list(product(tasks, num_observation, ['CNRE'], num_contrastive_parameters, gammas, num_blocks))
df_cnre = pd.DataFrame(data=experiments_cnre, columns=columns)
print("cnre: ", len(df_cnre))

experiments_nre = list(product(tasks, num_observation, ['NRE-B'], num_contrastive_parameters, [nre_gamma_default], num_blocks))
df_nre = pd.DataFrame(data=experiments_nre, columns=columns)
print("nre: ", len(df_nre))

df_ref = pd.concat([df_cnre, df_nre])
print("total: ", len(df_ref))
df_ref
df_ref.to_csv("experiment-reference.csv")

cnre:  2880
nre:  480
total:  3360


## loading files

In [6]:
files = [
    "metrics-joint.csv",
]
df_joint = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)
df_joint_wide, gpr_joint = wide_to_long(df_joint)

In [7]:
files = [
    "metrics-prior.csv",
]
df_prior = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)
df_prior_wide, gpr_prior = wide_to_long(df_prior)

In [8]:
files = [
    "metrics-bench.csv",
]
df_bench = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)
df_bench_wide, gpr_bench = wide_to_long(df_bench)

In [9]:
files = [
    "metrics-bench-extra.csv",
]
df_bench_extra = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)
df_bench_extra_wide, grp_bench_extra = wide_to_long(df_bench_extra)

In [10]:
files = [
    "metrics-sbibm.csv",
]
df_sbibm = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)

## what's missing?

In [11]:
def get_missing_experiments(expr: pd.DataFrame, ref: pd.DataFrame = df_ref) -> pd.DataFrame:
    merge_df = ref.merge(expr, how='left', indicator=True)
    merge_df = merge_df[merge_df['_merge'] == 'left_only']
    return merge_df.drop("_merge", axis=1)

In [12]:
df_joint_to_check = df_joint_wide.reset_index()[columns].drop_duplicates()
df_joint_missing = get_missing_experiments(df_joint_to_check)
print(df_joint_missing["task"].unique())
print(df_joint_missing["num_observation"].unique())
print(df_joint_missing["algorithm"].unique())
print(df_joint_missing["gamma"].unique())
print(df_joint_missing["num_blocks"].unique())
df_joint_missing

[]
[]
[]
[]
[]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks


In [13]:
df_prior_to_check = df_prior_wide.reset_index()[columns].drop_duplicates()
df_prior_missing = get_missing_experiments(df_prior_to_check)
print(df_prior_missing["task"].unique())
print(df_prior_missing["num_observation"].unique())
print(df_prior_missing["algorithm"].unique())
print(df_prior_missing["gamma"].unique())
print(df_prior_missing["num_blocks"].unique())
df_prior_missing

[]
[]
[]
[]
[]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks


In [14]:
df_bench_to_check = df_bench_wide.reset_index()[columns].drop_duplicates()
df_bench_missing = get_missing_experiments(df_bench_to_check)
print(df_bench_missing["task"].unique())
print(df_bench_missing["num_observation"].unique())
print(df_bench_missing["algorithm"].unique())
print(df_bench_missing["gamma"].unique())
print(df_bench_missing["num_blocks"].unique())
df_bench_missing

[]
[]
[]
[]
[]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks


In [15]:
df_bench_extra_to_check = df_bench_extra_wide.reset_index()[columns].drop_duplicates()
df_bench_extra_missing = get_missing_experiments(df_bench_extra_to_check)
print(df_bench_extra_missing["task"].unique())
print(df_bench_extra_missing["num_observation"].unique())
print(df_bench_extra_missing["algorithm"].unique())
print(df_bench_extra_missing["gamma"].unique())
print(df_bench_extra_missing["num_blocks"].unique())
df_bench_extra_missing

['slcp' 'two_moons' 'gaussian_mixture']
[ 1  2  3  4  5  6  7  8  9 10]
['CNRE' 'NRE-B']
[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02    inf]
[2 3]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks
0,slcp,1,CNRE,2,0.001,2
1,slcp,1,CNRE,2,0.001,3
2,slcp,1,CNRE,2,0.010,2
3,slcp,1,CNRE,2,0.010,3
4,slcp,1,CNRE,2,0.100,2
...,...,...,...,...,...,...
3355,gaussian_mixture,10,NRE-B,100,inf,3
3356,gaussian_mixture,10,NRE-B,150,inf,2
3357,gaussian_mixture,10,NRE-B,150,inf,3
3358,gaussian_mixture,10,NRE-B,200,inf,2


## what only has a single initialization?

In [16]:
df_joint_wide_counts = df_joint_wide.groupby(columns, as_index=False).size()
print(df_joint_wide_counts["size"].unique())
df_joint_wide_counts[df_joint_wide_counts["size"] == 1]
df_joint_wide_counts[df_joint_wide_counts["size"] > 1]

[1 2]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks,size
96,gaussian_mixture,1,NRE-B,2,inf,2,2
98,gaussian_mixture,1,NRE-B,10,inf,2,2
100,gaussian_mixture,1,NRE-B,25,inf,2,2
102,gaussian_mixture,1,NRE-B,50,inf,2,2
104,gaussian_mixture,1,NRE-B,75,inf,2,2
...,...,...,...,...,...,...,...
3350,two_moons,10,NRE-B,50,inf,2,2
3352,two_moons,10,NRE-B,75,inf,2,2
3354,two_moons,10,NRE-B,100,inf,2,2
3356,two_moons,10,NRE-B,150,inf,2,2


In [17]:
df_prior_wide_counts = df_prior_wide.groupby(columns, as_index=False).size()
print(df_prior_wide_counts["size"].unique())
df_prior_wide_counts[df_prior_wide_counts["size"] == 1]
df_prior_wide_counts[df_prior_wide_counts["size"] > 1]

[1]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks,size


In [18]:
df_bench_wide_counts = df_bench_wide.groupby(columns, as_index=False).size()
print(df_bench_wide_counts["size"].unique())
df_bench_wide_counts[df_bench_wide_counts["size"] == 1]
df_bench_wide_counts[df_bench_wide_counts["size"] > 1]

[3 2]


,task,num_observation,algorithm,num_contrastive_parameters,gamma,num_blocks,size
0,gaussian_mixture,1,CNRE,2,0.001,2,3
1,gaussian_mixture,1,CNRE,2,0.001,3,3
2,gaussian_mixture,1,CNRE,2,0.010,2,3
3,gaussian_mixture,1,CNRE,2,0.010,3,3
4,gaussian_mixture,1,CNRE,2,0.100,2,3
...,...,...,...,...,...,...,...
3355,two_moons,10,NRE-B,100,inf,3,3
3356,two_moons,10,NRE-B,150,inf,2,2
3357,two_moons,10,NRE-B,150,inf,3,3
3358,two_moons,10,NRE-B,200,inf,2,2


## measuring total compute time

In [19]:
a = df_joint["RT"].sum() + df_prior["RT"].sum() + df_bench["RT"].sum()
b = df_sbibm["RT"].sum()
print(a, b)
print(3 * a + b)

49639.71405131022 24157.288144938153
173076.43029886883
